<a href="https://colab.research.google.com/github/kainatquaderee/Ifooge/blob/main/Ifooge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Keep active

In [ ]:
# Keep Colab notebook active
from google.colab import output

code = '''
function KeepColabAlive() {
    setInterval(() => {
        console.log("Colab is running");
        document.querySelector("colab-toolbar-button#connect").click();
    }, 60000); // Change the interval if needed (60000 ms = 1 minute)
}
KeepColabAlive();
'''

output.eval_js(code)
print("KeepAlive script is running")

# version 1

In [ ]:
# Install necessary libraries
!pip install transformers diffusers gradio
!pip install git+https://github.com/TencentARC/GFPGAN.git
!pip install basicsr
!pip install git+https://github.com/xinntao/Real-ESRGAN.git

# Fix the import issue in basicsr library
!sed -i 's/from torchvision.transforms.functional_tensor import rgb_to_grayscale/from torchvision.transforms.functional import rgb_to_grayscale/' /usr/local/lib/python3.*/dist-packages/basicsr/data/degradations.py

# Keep Colab notebook active
from google.colab import output

code = '''
function KeepColabAlive() {
    setInterval(() => {
        console.log("Colab is running");
        document.querySelector("colab-toolbar-button#connect").click();
    }, 60000); // Change the interval if needed (60000 ms = 1 minute)
}
KeepColabAlive();
'''

output.eval_js(code)
print("KeepAlive script is running")

# Import necessary libraries
import torch
from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline
import gradio as gr
from gfpgan import GFPGANer
from PIL import Image
import numpy as np

# Load GFPGAN for face restoration
gfpgan = GFPGANer(
    model_path='https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth',
    upscale=1,
    arch='clean',
    channel_multiplier=2,
    bg_upsampler=None
)

# Function to fix faces
def fix_faces_func(image):
    image = np.array(image)
    _, _, restored_image = gfpgan.enhance(image, has_aligned=False, only_center_face=False)
    return Image.fromarray(restored_image)

# Function to load model from Hugging Face
def load_model(model_id):
    global pipe, img2img_pipe
    pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
    pipe.to("cuda")
    pipe.safety_checker = dummy_checker
    img2img_pipe = StableDiffusionImg2ImgPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
    img2img_pipe.to("cuda")
    img2img_pipe.safety_checker = dummy_checker

# Remove the safety checker
def dummy_checker(images, **kwargs):
    return images, [False] * len(images)

# Function to generate images
def generate_image(model_id, prompt="", negative_prompt="lowres, bad quality, deformed, disfigured", num_inference_steps=50, guidance_scale=7.5, fix_faces=False):
    load_model(model_id)
    with torch.autocast("cuda"):
        # Generate initial image
        image = pipe(prompt, negative_prompt=negative_prompt, num_inference_steps=num_inference_steps, guidance_scale=guidance_scale).images[0]

    if fix_faces:
        image = fix_faces_func(image)

    return image

# Function to generate images from an input image
def generate_img2img(model_id, init_image, prompt="", negative_prompt="lowres, bad quality, deformed, disfigured", num_inference_steps=50, guidance_scale=7.5, denoising_strength=0.75, fix_faces=False):
    load_model(model_id)
    with torch.autocast("cuda"):
        # Generate image from input image
        image = img2img_pipe(prompt, negative_prompt=negative_prompt, init_image=init_image, num_inference_steps=num_inference_steps, guidance_scale=guidance_scale, strength=denoising_strength).images[0]

    if fix_faces:
        image = fix_faces_func(image)

    return image

# Custom CSS for a black modern theme
css = """
body {background-color: #121212; color: #ffffff;}
.gradio-container {background-color: #121212 !important; border-color: #333333 !important;}
.gr-button {background-color: #333333 !important; color: #ffffff !important; border-color: #333333 !important;}
.gr-button:hover {background-color: #444444 !important; border-color: #444444 !important;}
.gr-textbox {background-color: #1f1f1f !important; color: #ffffff !important; border-color: #333333 !important;}
.gr-slider {color: #ffffff !important;}
.gr-checkbox label {color: #ffffff !important;}
"""

# Create a Gradio interface with two tabs
iface = gr.Blocks(css=css)

with iface:
    gr.Markdown("# ifooge\nGenerate images using Stable Diffusion and GFPGAN. By Kaiden Quinart.")

    with gr.Tabs():
        with gr.TabItem("Text-to-Image"):
            gr.Markdown("## Text-to-Image")
            with gr.Row():
                with gr.Column():
                    model_id = gr.Textbox(label="Hugging Face Model ID", value="stablediffusionapi/realistic-vision-vae-5")
                    prompt = gr.Textbox(label="Prompt Details")
                    negative_prompt = gr.Textbox(label="Negative Prompt", value="lowres, bad quality, deformed, disfigured")
                    num_inference_steps = gr.Slider(label="Inference Steps", minimum=1, maximum=100, value=50, step=1)
                    guidance_scale = gr.Slider(label="Guidance Scale", minimum=0.1, maximum=20.0, value=7.5)
                    fix_faces = gr.Checkbox(label="Fix Faces")
                with gr.Column():
                    generate_button = gr.Button("Generate Image")
                    output_image = gr.Image(label="Generated Image")

            generate_button.click(generate_image, inputs=[model_id, prompt, negative_prompt, num_inference_steps, guidance_scale, fix_faces], outputs=output_image)

        with gr.TabItem("Image-to-Image"):
            gr.Markdown("## Image-to-Image")
            with gr.Row():
                with gr.Column():
                    model_id_img2img = gr.Textbox(label="Hugging Face Model ID", value="stablediffusionapi/realistic-vision-vae-5")
                    init_image = gr.Image(label="Initial Image")
                    prompt_img2img = gr.Textbox(label="Prompt Details")
                    negative_prompt_img2img = gr.Textbox(label="Negative Prompt", value="lowres, bad quality, deformed, disfigured")
                    num_inference_steps_img2img = gr.Slider(label="Inference Steps", minimum=1, maximum=100, value=50, step=1)
                    guidance_scale_img2img = gr.Slider(label="Guidance Scale", minimum=0.1, maximum=20.0, value=7.5)
                    denoising_strength = gr.Slider(label="Denoising Strength", minimum=0.0, maximum=1.0, value=0.75)
                    fix_faces_img2img = gr.Checkbox(label="Fix Faces")
                with gr.Column():
                    generate_button_img2img = gr.Button("Generate Image")
                    output_image_img2img = gr.Image(label="Generated Image")

            generate_button_img2img.click(generate_img2img, inputs=[model_id_img2img, init_image, prompt_img2img, negative_prompt_img2img, num_inference_steps_img2img, guidance_scale_img2img, denoising_strength, fix_faces_img2img], outputs=output_image_img2img)

# Launch the interface with debug mode enabled
iface.launch(share=True, debug=True)

SyntaxError: invalid syntax (<ipython-input-1-ea0c96222b63>, line 107)

# version 2

In [ ]:
# Install necessary libraries
!pip install transformers diffusers gradio
!pip install git+https://github.com/TencentARC/GFPGAN.git
!pip install basicsr
!pip install git+https://github.com/xinntao/Real-ESRGAN.git

# Fix the import issue in basicsr library
!sed -i 's/from torchvision.transforms.functional_tensor import rgb_to_grayscale/from torchvision.transforms.functional import rgb_to_grayscale/' /usr/local/lib/python3.*/dist-packages/basicsr/data/degradations.py

# Keep Colab notebook active
from google.colab import output

code = '''
function KeepColabAlive() {
    setInterval(() => {
        console.log("Colab is running");
        document.querySelector("colab-toolbar-button#connect").click();
    }, 60000); // Change the interval if needed (60000 ms = 1 minute)
}
KeepColabAlive();
'''

output.eval_js(code)
print("KeepAlive script is running")

# Import necessary libraries
import torch
from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline
import gradio as gr
from gfpgan import GFPGANer
from PIL import Image
import numpy as np

# Load GFPGAN for face restoration
gfpgan = GFPGANer(
    model_path='https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth',
    upscale=1,
    arch='clean',
    channel_multiplier=2,
    bg_upsampler=None
)

# Function to fix faces
def fix_faces_func(image):
    image = np.array(image)
    _, _, restored_image = gfpgan.enhance(image, has_aligned=False, only_center_face=False)
    return Image.fromarray(restored_image)

# Function to load model from Hugging Face
def load_model(model_id):
    global pipe, img2img_pipe
    pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
    pipe.to("cuda")
    pipe.safety_checker = dummy_checker
    img2img_pipe = StableDiffusionImg2ImgPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
    img2img_pipe.to("cuda")
    img2img_pipe.safety_checker = dummy_checker

# Remove the safety checker
def dummy_checker(images, **kwargs):
    return images, [False] * len(images)

# Function to generate images
def generate_image(model_id, prompt="", negative_prompt="lowres, bad quality, deformed, disfigured", num_inference_steps=50, guidance_scale=7.5, fix_faces=False, seed=None, height=512, width=512):
    load_model(model_id)
    if seed is None:
        seed = np.random.randint(0, 2**32 - 1)
    generator = torch.manual_seed(seed)
    with torch.autocast("cuda"):
        # Generate initial image
        image = pipe(prompt, negative_prompt=negative_prompt, num_inference_steps=num_inference_steps, guidance_scale=guidance_scale, height=height, width=width, generator=generator).images[0]

    if fix_faces:
        image = fix_faces_func(image)

    return image, seed

# Function to generate images from an input image
def generate_img2img(model_id, init_image, prompt="", negative_prompt="lowres, bad quality, deformed, disfigured", num_inference_steps=50, guidance_scale=7.5, denoising_strength=0.75, fix_faces=False, seed=None, height=512, width=512):
    load_model(model_id)
    if seed is None:
        seed = np.random.randint(0, 2**32 - 1)
    generator = torch.manual_seed(seed)
    init_image = init_image.convert("RGB")
    init_image = init_image.resize((width, height))
    with torch.autocast("cuda"):
        # Generate image from input image
        image = img2img_pipe(prompt, negative_prompt=negative_prompt, init_image=init_image, num_inference_steps=num_inference_steps, guidance_scale=guidance_scale, strength=denoising_strength, generator=generator).images[0]

    if fix_faces:
        image = fix_faces_func(image)

    return image, seed

# Custom CSS for a black modern theme
css = """
body {background-color: #121212; color: #ffffff;}
.gradio-container {background-color: #121212 !important; border-color: #333333 !important;}
.gr-button {background-color: #333333 !important; color: #ffffff !important; border-color: #333333 !important;}
.gr-button:hover {background-color: #444444 !important; border-color: #444444 !important;}
.gr-textbox {background-color: #1f1f1f !important; color: #ffffff !important; border-color: #333333 !important;}
.gr-slider {color: #ffffff !important;}
.gr-checkbox label {color: #ffffff !important;}
"""

# Create a Gradio interface with two tabs
iface = gr.Blocks(css=css)

with iface:
    gr.Markdown("# ifooge\nGenerate images using Stable Diffusion and GFPGAN. By Kaiden Quinart.")

    with gr.Tabs():
        with gr.TabItem("Text-to-Image"):
            gr.Markdown("## Text-to-Image")
            with gr.Row():
                with gr.Column():
                    model_id = gr.Textbox(label="Hugging Face Model ID", value="stablediffusionapi/realistic-vision-vae-5")
                    prompt = gr.Textbox(label="Prompt Details")
                    negative_prompt = gr.Textbox(label="Negative Prompt", value="lowres, bad quality, deformed, disfigured")
                    num_inference_steps = gr.Slider(label="Inference Steps", minimum=1, maximum=100, value=50, step=1)
                    guidance_scale = gr.Slider(label="Guidance Scale", minimum=0.1, maximum=20.0, value=7.5)
                    fix_faces = gr.Checkbox(label="Fix Faces")
                    seed = gr.Number(label="Seed", value=None, interactive=True)
                    height = gr.Slider(label="Height", minimum=256, maximum=1024, value=512, step=64)
                    width = gr.Slider(label="Width", minimum=256, maximum=1024, value=512, step=64)
                with gr.Column():
                    generate_button = gr.Button("Generate Image")
                    output_image = gr.Image(label="Generated Image")
                    output_seed = gr.Textbox(label="Seed Used")

            generate_button.click(generate_image, inputs=[model_id, prompt, negative_prompt, num_inference_steps, guidance_scale, fix_faces, seed, height, width], outputs=[output_image, output_seed])

        with gr.TabItem("Image-to-Image"):
            gr.Markdown("## Image-to-Image")
            with gr.Row():
                with gr.Column():
                    model_id_img2img = gr.Textbox(label="Hugging Face Model ID", value="stablediffusionapi/realistic-vision-vae-5")
                    init_image = gr.Image(label="Initial Image")
                    prompt_img2img = gr.Textbox(label="Prompt Details")
                    negative_prompt_img2img = gr.Textbox(label="Negative Prompt", value="lowres, bad quality, deformed, disfigured")
                    num_inference_steps_img2img = gr.Slider(label="Inference Steps", minimum=1, maximum=100, value=50, step=1)
                    guidance_scale_img2img = gr.Slider(label="Guidance Scale", minimum=0.1, maximum=20.0, value=7.5)
                    denoising_strength = gr.Slider(label="Denoising Strength", minimum=0.0, maximum=1.0, value=0.75)
                    fix_faces_img2img = gr.Checkbox(label="Fix Faces")
                    seed_img2img = gr.Number(label="Seed", value=None, interactive=True)
                    height_img2img = gr.Slider(label="Height", minimum=256, maximum=1024, value=512, step=64)
                    width_img2img = gr.Slider(label="Width", minimum=256, maximum=1024, value=512, step=64)
                with gr.Column():
                    generate_button_img2img = gr.Button("Generate Image")
                    output_image_img2img = gr.Image(label="Generated Image")
                    output_seed_img2img = gr.Textbox(label="Seed Used")

            generate_button_img2img.click(generate_img2img, inputs=[model_id_img2img, init_image, prompt_img2img, negative_prompt_img2img, num_inference_steps_img2img, guidance_scale_img2img, denoising_strength, fix_faces_img2img, seed_img2img, height_img2img, width_img2img], outputs=[output_image_img2img, output_seed_img2img])

# Launch the interface with debug mode enabled
iface.launch(share=True, debug=True)

# animatediff

In [ ]:
# Install necessary libraries
!pip install transformers diffusers gradio
!pip install git+https://github.com/TencentARC/GFPGAN.git
!pip install basicsr
!pip install git+https://github.com/xinntao/Real-ESRGAN.git

# Fix the import issue in basicsr library
!sed -i 's/from torchvision.transforms.functional_tensor import rgb_to_grayscale/from torchvision.transforms.functional import rgb_to_grayscale/' /usr/local/lib/python3.*/dist-packages/basicsr/data/degradations.py

# Keep Colab notebook active
from google.colab import output

code = '''
function KeepColabAlive() {
    setInterval(() => {
        console.log("Colab is running");
        document.querySelector("colab-toolbar-button#connect").click();
    }, 60000); // Change the interval if needed (60000 ms = 1 minute)
}
KeepColabAlive();
'''

output.eval_js(code)
print("KeepAlive script is running")

# Import necessary libraries
import torch
from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline, AnimateDiffPipeline, DDIMScheduler, MotionAdapter
import gradio as gr
from gfpgan import GFPGANer
from PIL import Image
import numpy as np
from diffusers.utils import export_to_gif

# Load GFPGAN for face restoration
gfpgan = GFPGANer(
    model_path='https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth',
    upscale=1,
    arch='clean',
    channel_multiplier=2,
    bg_upsampler=None
)

# Function to fix faces
def fix_faces_func(image):
    image = np.array(image)
    _, _, restored_image = gfpgan.enhance(image, has_aligned=False, only_center_face=False)
    return Image.fromarray(restored_image)

# Function to load model from Hugging Face
def load_model(model_id):
    global pipe, img2img_pipe
    pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
    pipe.to("cuda")
    pipe.safety_checker = dummy_checker
    img2img_pipe = StableDiffusionImg2ImgPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
    img2img_pipe.to("cuda")
    img2img_pipe.safety_checker = dummy_checker

# Remove the safety checker
def dummy_checker(images, **kwargs):
    return images, [False] * len(images)

# Function to generate images
def generate_image(model_id, prompt="", negative_prompt="lowres, bad quality, deformed, disfigured", num_inference_steps=50, guidance_scale=7.5, fix_faces=False, seed=None, height=512, width=512):
    load_model(model_id)
    if seed is None:
        seed = np.random.randint(0, 2**32 - 1)
    generator = torch.manual_seed(seed)
    with torch.autocast("cuda"):
        # Generate initial image
        image = pipe(prompt, negative_prompt=negative_prompt, num_inference_steps=num_inference_steps, guidance_scale=guidance_scale, height=height, width=width, generator=generator).images[0]

    if fix_faces:
        image = fix_faces_func(image)

    return image, seed

# Function to generate images from an input image
def generate_img2img(model_id, init_image, prompt="", negative_prompt="lowres, bad quality, deformed, disfigured", num_inference_steps=50, guidance_scale=7.5, denoising_strength=0.75, fix_faces=False, seed=None, height=512, width=512):
    load_model(model_id)
    if seed is None:
        seed = np.random.randint(0, 2**32 - 1)
    generator = torch.manual_seed(seed)
    init_image = init_image.convert("RGB")
    init_image = init_image.resize((width, height))
    with torch.autocast("cuda"):
        # Generate image from input image
        image = img2img_pipe(prompt, negative_prompt=negative_prompt, init_image=init_image, num_inference_steps=num_inference_steps, guidance_scale=guidance_scale, strength=denoising_strength, generator=generator).images[0]

    if fix_faces:
        image = fix_faces_func(image)

    return image, seed

# Function to generate text-to-video
def generate_text2video(prompt, negative_prompt="bad quality, worse quality", num_frames=16, guidance_scale=7.5, num_inference_steps=25, seed=None):
    if seed is None:
        seed = np.random.randint(0, 2**32 - 1)
    generator = torch.manual_seed(seed)

    adapter = MotionAdapter.from_pretrained("guoyww/animatediff-motion-adapter-v1-5-2", torch_dtype=torch.float16)
    model_id = "SG161222/Realistic_Vision_V5.1_noVAE"
    pipe = AnimateDiffPipeline.from_pretrained(model_id, motion_adapter=adapter, torch_dtype=torch.float16)
    scheduler = DDIMScheduler.from_pretrained(
        model_id,
        subfolder="scheduler",
        clip_sample=False,
        timestep_spacing="linspace",
        beta_schedule="linear",
        steps_offset=1,
    )
    pipe.scheduler = scheduler

    pipe.enable_vae_slicing()
    pipe.enable_model_cpu_offload()

    output = pipe(
        prompt=prompt,
        negative_prompt=negative_prompt,
        num_frames=num_frames,
        guidance_scale=guidance_scale,
        num_inference_steps=num_inference_steps,
        generator=generator,
    )

    frames = output.frames[0]
    gif_path = "animation.gif"
    export_to_gif(frames, gif_path)

    return gif_path, seed

# Custom CSS for a black modern theme
css = """
body {background-color: #121212; color: #ffffff;}
.gradio-container {background-color: #121212 !important; border-color: #333333 !important;}
.gr-button {background-color: #333333 !important; color: #ffffff !important; border-color: #333333 !important;}
.gr-button:hover {background-color: #444444 !important; border-color: #444444 !important;}
.gr-textbox {background-color: #1f1f1f !important; color: #ffffff !important; border-color: #333333 !important;}
.gr-slider {color: #ffffff !important;}
.gr-checkbox label {color: #ffffff !important;}
"""

# Create a Gradio interface with three tabs
iface = gr.Blocks(css=css)

with iface:
    gr.Markdown("# ifooge\nGenerate images and videos using Stable Diffusion, GFPGAN, and AnimateDiff. By Kaiden Quinart.")

    with gr.Tabs():
        with gr.TabItem("Text-to-Image"):
            gr.Markdown("## Text-to-Image")
            with gr.Row():
                with gr.Column():
                    model_id = gr.Textbox(label="Hugging Face Model ID", value="stablediffusionapi/realistic-vision-vae-5")
                    prompt = gr.Textbox(label="Prompt Details")
                    negative_prompt = gr.Textbox(label="Negative Prompt", value="lowres, bad quality, deformed, disfigured")
                    num_inference_steps = gr.Slider(label="Inference Steps", minimum=1, maximum=100, value=50, step=1)
                    guidance_scale = gr.Slider(label="Guidance Scale", minimum=0.1, maximum=20.0, value=7.5)
                    fix_faces = gr.Checkbox(label="Fix Faces")
                    seed = gr.Number(label="Seed", value=None, interactive=True)
                    height = gr.Slider(label="Height", minimum=256, maximum=1024, value=512, step=64)
                    width = gr.Slider(label="Width", minimum=256, maximum=1024, value=512, step=64)
                with gr.Column():
                    generate_button = gr.Button("Generate Image")
                    output_image = gr.Image(label="Generated Image")
                    output_seed = gr.Textbox(label="Seed Used")

            generate_button.click(generate_image, inputs=[model_id, prompt, negative_prompt, num_inference_steps, guidance_scale, fix_faces, seed, height, width], outputs=[output_image, output_seed])

        with gr.TabItem("Image-to-Image"):
            gr.Markdown("## Image-to-Image")
            with gr.Row():
                with gr.Column():
                    model_id_img2img = gr.Textbox(label="Hugging Face Model ID", value="stablediffusionapi/realistic-vision-vae-5")
                    init_image = gr.Image(label="Initial Image")
                    prompt_img2img = gr.Textbox(label="Prompt Details")
                    negative_prompt_img2img = gr.Textbox(label="Negative Prompt", value="lowres, bad quality, deformed, disfigured")
                    num_inference_steps_img2img = gr.Slider(label="Inference Steps", minimum=1, maximum=100, value=50, step=1)
                    guidance_scale_img2img = gr.Slider(label="Guidance Scale", minimum=0.1, maximum=20.0, value=7.5)
                    denoising_strength = gr.Slider(label="Denoising Strength", minimum=0.0, maximum=1.0, value=0.75)
                    fix_faces_img2img = gr.Checkbox(label="Fix Faces")
                    seed_img2img